# Stride Scheduler Simulator With Arrivals

## Introduction

This notebooks implements a simple lottery scheduler simulator.  By changing the *number of time steps*, the *set of processes*, the *number of tickets per process*, and the *arrival times of processes* (on lines 14 through 19), one can explore how various settings for those parameters affect the generated schedules.

## Running a cell
If you click on the cell below, you can run it by hitting the *Play* button to the left of it.

## Output
The output that is generated from the cell is a graph containing *time steps* on the x-axis, *percentage of CPU use* on the y-axis, with a line plot for each process.

In [ ]:
def get_associated_process(pass_values_arg):
    min_index = 0
    min_value = pass_values_arg[min_index]
    for index in range(1,len(pass_values_arg)):
        if (pass_values_arg[index] < min_value):
            min_index = index
            min_value = pass_values_arg[min_index]
    return min_index

import copy
import matplotlib.pyplot as plotter

# Number of timesteps to emulate in scheduling
number_of_timesteps = 100

# Each process has a name and an associated number of tickets 
process_names = ["A", "B", "C"]
process_tickets = {"A":100,"B":100,"C":100}
arrival_times = {0:["A","B","C"]}

# These are lists holding the names and tickets for the processes 
# that have already arrived and need to be scheduled
active_process_names = []
active_process_tickets = []

# This is the magic number to use to compute strides (it can be any value)
magic_number = 6000


# Summary information about the simulator
number_of_processes = len(process_names)
print("Number of processes in simulation: ", number_of_processes)
print("Simulating ", number_of_timesteps, " timesteps")

# Schedule is initially empty
schedule = [""] * number_of_timesteps

# Pass values are initially an empty list, will be added to when processes arrive
pass_values = []

# Initially, no processes are active
number_of_active_processes = 0

# For each timestep, 
# - Handle any process arrivals
# - Choose next process based on pass values
# - Record the chosen process in the schedule
for timestep in range(number_of_timesteps):
    # See if anyone arrives, if so, add them to the pool and determine number of tickets in system
    if timestep in arrival_times:
        # See if anyone arrives, if so, add them to the active processes and 
        # determine total number of tickets in system; also set pass values
        # to 0 for any new processes
        print("New arrivals at time", timestep)
        arrival_list = arrival_times[timestep]
        for index in range(len(arrival_list)):
            process_name = arrival_list[index]
            active_process_names.append(process_name)
            active_process_tickets.append(process_tickets[process_name])
            pass_values.append(0.0)
        number_of_active_processes = len(active_process_names)
        # (Re)Compute stride values for active processes
        process_strides = []
        for process in range(number_of_active_processes):
            process_strides.append(magic_number / active_process_tickets[process])        

    # if there are any processes active, then schedule one of them
    if len(active_process_names) > 0:
        chosen_process = get_associated_process(pass_values)
        schedule[timestep] = chosen_process
        pass_values[chosen_process] = pass_values[chosen_process] + process_strides[chosen_process]
    else:
        schedule[timestep] = "IDLE"

# Uncomment this if want to see the actual schedule (may be long!)
#print("Schedule generated:")
#for timestep in range(number_of_timesteps):
    #print(timestep," ", schedule[timestep])

# Summarize the schedule to graph it
schedule_counts = []
schedule_percentages = []
for process in range(number_of_processes):
    schedule_counts.append([0]*number_of_timesteps)
    schedule_percentages.append([0]*number_of_timesteps)
    
for timestep in range(number_of_timesteps):
    scheduled_process = schedule[timestep]
    for process in range(number_of_processes):
        if timestep == 0:
            previous_count = 0
        else:
            previous_count = schedule_counts[process][timestep-1]
        schedule_counts[process][timestep] = previous_count
        if (process == scheduled_process):
            schedule_counts[process][timestep] = previous_count + 1
            
for timestep in range(number_of_timesteps):
    sum = 0
    for process in range(number_of_processes):
        sum = sum + schedule_counts[process][timestep]
    for process in range(number_of_processes):
        schedule_percentages[process][timestep] = schedule_counts[process][timestep] / (sum*1.0)

# Graph the summary
%matplotlib inline 
for process in range(number_of_processes):
    plotter.plot(schedule_percentages[process],label="Process " + process_names[process])
plotter.yticks([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
plotter.grid(axis='both')
plotter.legend()


## Experimentation

Try these experiments with the code and be ready to discuss what you are seeing:

* With all processes in the system, explore smaller (`10`, `100`) and larger (`1000`,`10000`) values for `number_of_timesteps` and see if the proportions converge to what you expect.
* Add another process `D`, with an equal number of tickets as processes `A`,`B`, and `C` and also arriving at time `0`. Predict what the proportions should be in that case, and then run the simulation to check your prediction
* With the four processes (`A`,`B`,`C`,`D`), all arriving at time `0`, set the tickets so that process `A` obtains approximately 80% of the CPU time and the rest obtain an equal share of the remaining 20% of the CPU time. Run the simulation to verify if your settings work.
* Set the `number_of_timesteps` to 500 and set process `D` to arrive at timestep 100 with a priority equal to that of process `A`.  To do that, follow the rules below. Then, think through what the % of CPU time that each process should receive is and whether the actual schedule reflects that.
    * Set the `arrival_times` to be equal to `{0:["A","B","C"], 100:["D"]}`
    * Set the number of tickets for process `D` to be the same as you had for process `A` in the previous experiment.